<a href="https://colab.research.google.com/github/nicolazilio0/deepRiccy/blob/main/static_baseline%2Bstable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep riccy project


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

! tar -zxf /content/drive/MyDrive/refcocog.tar.gz
! pip3 install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt 
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

local_path = '/content/refcocog/images/'
local_annotations = '/content/refcocog/annotations/'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 592.7/592.7 kB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-hsao009h
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-hsao009h
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369370 sha256=a74b7f03b1dc41aabc7bbc7bc4b991ed8d5e1e231

## Env creation


In [ ]:
local_path = 'refcocog/images/'
local_annotations = 'refcocog/annotations/'

In [6]:
! pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 34.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350809 sha256=0e22c7bf622dd7941aa726ed518d7324e1e64a8dd83e8480b907a693b1c625ae
  Stored in directory: /root/.cache/pip/wheels/79/29/1c/234cae4632803c2ba4a76a71a679eb1383cf590775714e2a21
Successfully built emoji


In [7]:
# imports

import json
import pickle
import torch
from PIL import Image, ImageDraw
from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from pkg_resources import packaging
import clip
import numpy as np

import os
import skimage
import IPython.display

from collections import OrderedDict
import torch
import torchmetrics as tm
import torchvision
from torchvision import ops
import stanza

In [8]:
stanza.download('en',package='partut')
nlp = stanza.Pipeline(lang='en')

INFO:stanza:Downloading these customized packages for language: en (English)...
| Processor       | Package  |
------------------------------
| tokenize        | partut   |
| mwt             | partut   |
| pos             | partut   |
| lemma           | partut   |
| depparse        | partut   |
| backward_charlm | 1billion |
| pretrain        | partut   |
| forward_charlm  | 1billion |



INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


## Baseline


### Utilities


In [9]:
def remove_of(sentence):
    if "side of" in sentence:
        index = sentence.find("side of")
        sentence = sentence[index+7:]
        return sentence
    if "handle of" in sentence:
        index = sentence.find("handle of")
        sentence = sentence[index+9:]
    if "bunch of" in sentence:
        index = sentence.find("bunch of")
        sentence = sentence[index+8:]
        return sentence
    if "corner of" in sentence:
        index = sentence.find("corner of")
        sentence = sentence[index+9:]
        return sentence
    if "end of" in sentence:
        index = sentence.find("end of")
        sentence = sentence[index+6:]
        return sentence
    if "half of" in sentence:
        index = sentence.find("half of")
        sentence = sentence[index+7:]
        return sentence
    if "edge of" in sentence:
        index = sentence.find("edge of")
        sentence = sentence[index+7:]
        return sentence
    if "back of" in sentence:
        index = sentence.find("back of")
        sentence = sentence[index+7:]
        return sentence
    if "smaller of" in sentence:
        index = sentence.find("smaller of")
        sentence = sentence[index+10:]
        return sentence
    if "piece of" in sentence:
        index = sentence.find("piece of")
        sentence = sentence[index+8:]
        return sentence
    if "wing of" in sentence:
        index = sentence.find("wing of")
        sentence = sentence[index+7:]

    if "front of" in sentence:
        index = sentence.find("front of")
        sentence = sentence[index+8:]
        return sentence
    if "back side of" in sentence:
        index = sentence.find("back side of")
        sentence = sentence[index+12:]
        return sentence
    if "pile of" in sentence:
        index = sentence.find("pile of")
        sentence = sentence[index+7:]
        return sentence
    if "pair of" in sentence:
        index = sentence.find("pair of")
        sentence = sentence[index+7:]
        return sentence
    if "pieces of" in sentence:
        index = sentence.find("pieces of")
        sentence = sentence[index+9:]
        return sentence
    if "intersection of" in sentence:
        index = sentence.find("intersection of")
        sentence = sentence[index+15:]
        return sentence
    if "middle of" in sentence:
        index = sentence.find("middle of")
        sentence = sentence[index+9:]
        return sentence
    if "patch of" in sentence:
        index = sentence.find("patch of")
        sentence = sentence[index+8:]
        return sentence
    if "couple of" in sentence:
        index = sentence.find("couple of")
        sentence = sentence[index+8:]
        return sentence
    if "slice of" in sentence:
        index = sentence.find("slice of")
        sentence = sentence[index+8:]
        return sentence
    if "tallest of" in sentence:
        index = sentence.find("tallest of")
        sentence = sentence[index+10:]
        return sentence
    if "kind of" in sentence:
        index = sentence.find("kind of")
        sentence = sentence[index+7:]
        return sentence
    if "that is" in sentence:
        index = sentence.find("that is")
        sentence = sentence[:index]
        return sentence
    if "part of" in sentence:
        index = sentence.find("part of")
        sentence = sentence[index+7:]
        return sentence
    if "corner of" in sentence:
        index = sentence.find("corner of")
        sentence = sentence[index+9:]
        return sentence
    if "half of" in sentence:
        index = sentence.find("half of")
        sentence = sentence[index+7:]
        return sentence
    if "top of" in sentence:
        index = sentence.find("top of")
        sentence = sentence[index+6:]
        return sentence
    return sentence

In [10]:
def sent_stanza_processing(sentence):
    sentence = sentence.lower()
    if sentence.startswith('there is '):
        sentence = 'the ' + sentence[9:]
    if sentence.startswith('this is '):
        sentence = sentence[9:]
    sentence = remove_of(sentence)

    nlp_sent = sentence.lower()

    # put the sentence in lower case
    # if the sentence does not start with "a" or "the" insert it
    x = nlp_sent.split(" ")
    if (x[0] != "the" and x[0] != "a"):
        nlp_sent = "the " + nlp_sent

    doc = nlp(nlp_sent)
    # print nlp dependencies
    # doc.sentences[0].print_dependencies()
    # print(input["sentences"]["raw"])
    root = ''
    phrase_upos = []
    # get heads of words
    heads = [sent.words[word.head -
                        1].text for sent in doc.sentences for word in sent.words]
    for sent in doc.sentences:
        for word in sent.words:
            # if it is a verbal phrase then take the nominal subject of the phrase
            if (word.deprel == 'nsubj' or word.deprel == 'nsubj:pass'):
                root = word.text
                return word.text
                # print(word.text)
                break
            # print(word)
            phrase_upos.append(word)
            # else take the root of the phrase
            if (word.head == 0):
                # print(word.text)
                return word.text
                # root=word.text
                # if the root is a verb
                if (word.upos == 'VERB'):
                    for w in reversed(phrase_upos):
                        # go back until you get a noun
                        if (w.upos == 'NN'):
                            return word.text
                            # print(w.text)

In [11]:
def get_root(yolo_output, sentence, model, yolo):
    root = sent_stanza_processing(sentence)
    # print(root)
    prompt_tokens = clip.tokenize(
        root, context_length=77, truncate=True).cuda()
    with torch.no_grad():
        prompt_features = model.encode_text(prompt_tokens).float()

    names = []
    for a in range(len(yolo_output.xyxy[0])):
        class_index = int(yolo_output.pred[0][a][5])
        label = yolo.names[class_index]
        names.append(label)
    tokens = clip.tokenize(names, context_length=77, truncate=True).cuda()
    with torch.no_grad():
        classes_features = model.encode_text(tokens).float()
    prompt_features /= prompt_features.norm(dim=-1, keepdim=True)
    classes_features /= classes_features.norm(dim=-1, keepdim=True)
    prompt_similarity = classes_features.cpu().numpy() @ prompt_features.cpu().numpy().T
    if prompt_similarity.shape[0] == 0:
        return "empty"
    rappresentation = np.argmax(prompt_similarity)

    interested_class = names[rappresentation]
    return interested_class

In [12]:
# given an image pad the image to fit the dimension of clip and fill the padding with the median color of the image

def padd_image(img):
    avg_color_per_row = np.average(img, axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    old_image_width, old_image_height = img.size
    # create new image of desired size and color (blue) for padding
    if (old_image_height > 224 or old_image_width > 224):
        if (old_image_height > old_image_width):
            new_image_width = old_image_height
            new_image_height = old_image_height

        else:
            new_image_width = old_image_width
            new_image_height = old_image_width

        color = avg_color
        # color = (255,0,255)

        result = np.full((new_image_height, new_image_width, 3),
                         color, dtype=np.uint8)

        # compute center offset
        x_center = (new_image_width - old_image_width) // 2
        y_center = (new_image_height - old_image_height) // 2

        # copy img image into center of result image
        result[y_center:y_center+old_image_height,
               x_center:x_center+old_image_width] = img

    else:
        new_image_width = 224
        new_image_height = 224

        color = avg_color
        result = np.full((new_image_height, new_image_width, 3),
                         color, dtype=np.uint8)
        # compute center offset
        x_center = (new_image_width - old_image_width) // 2
        y_center = (new_image_height - old_image_height) // 2

        # copy img image into center of result image
        result[y_center:y_center+old_image_height,
               x_center:x_center+old_image_width] = img

    img = Image.fromarray(result)
    img = img.resize((224, 224))
    return img

In [13]:
# remove the id in the image name string
def split_string(string):
    string = string.split("_")
    string = string[:-1]
    string = "_".join(string)
    append = ".jpg"
    string = string + append

    return string

In [14]:
# convert yolo format bbox into standard type
def convert_bbox(bbox, img):
    x1, y1, width, height = bbox
    x2, y2 = x1 + width, y1 + height

    # Verify coordinates
    if x1 < 0 or y1 < 0 or x2 > img.width or y2 > img.height:
        print("Bounding box fuori dai limiti dell'immagine!")
    else:
        return x1, y1, x2, y2

In [15]:
# yolo bbox include class and precision, drop them
def convert_yolo_bbox(bbox):
    return bbox[:4]

In [16]:
# test model on a number N of images and print metrics each tenth of the images
def test_on_n_images(model, dataset, N):
    model.reset_metrics()

    for i in range(0, N):
        input = dataset[i]
        image_path = split_string(input["file_name"])
        sentence = input["sentences"]["raw"]
        gt = input["bbox"]
        original_img = Image.open(local_path + image_path).convert("RGB")
        # print img dimensions and box coordinates
        bbox, img = model.evaluate(image_path, sentence, gt, original_img, i)

        if i % (N/10) == 0:
            print("Iteration: ", i)
            print(model.get_metrics())

In [27]:
# Validation function if given a dataframe runs the model parameter for the whole dataframe and prints the metrics
def validate(model, dataframe):
    model.reset_metrics()
    for i in range(0, 500):
    #for i in range(0, len(dataframe)):
        input = dataframe.iloc[i]
        image_path = split_string(input["file_name"])
        sentence = input["sentences"]["raw"]
        gt = input["bbox"]
        original_img = Image.open(local_path + image_path).convert("RGB")
        # print img dimensions and box coordinates
        model.evaluate(image_path, sentence, gt, original_img,i)

    print(model.get_metrics())

In [52]:
def process_stable_images(index,clip_model):
  stable_input1=Image.open("drive/MyDrive/stable_diffusion_images/stable_diffusion_"+str(index)+"_1.jpg")
  stable_input2=Image.open("drive/MyDrive/stable_diffusion_images/stable_diffusion_"+str(index)+"_2.jpg")
  stable_input3=Image.open("drive/MyDrive/stable_diffusion_images/stable_diffusion_"+str(index)+"_3.jpg")
  with torch.no_grad():
    convert_tensor = transforms.ToTensor()

    stable_input1=stable_input1.resize([224,224])   
    stable_input2=stable_input2.resize([224,224])   
    stable_input3=stable_input3.resize([224,224]) 

    image_stable1=torch.tensor(np.stack(convert_tensor(stable_input1))).cuda()   
    image_stable2=torch.tensor(np.stack(convert_tensor(stable_input2))).cuda()   
    image_stable3=torch.tensor(np.stack(convert_tensor(stable_input3))).cuda()   
    
    img_tens = torch.stack([image_stable1,image_stable2,image_stable3])
    
    stable_features= clip_model.encode_image(img_tens).float()

  return stable_features



### Metrics


In [19]:
# definition of recall metric
class Recall:
    def __init__(self):
        self.true_positives = 0
        self.false_negatives = 0

    def update(self, correct):
        if correct:
            self.true_positives += 1
        else:
            self.false_negatives += 1

    def compute(self):
        return self.true_positives / (self.true_positives + self.false_negatives)

    def reset(self):
        self.true_positives = 0
        self.false_negatives = 0

In [20]:
class Metrics:
    def __init__(self, model, trasform, treshold):
        self.transform = trasform
        # initialize torch tensor
        self.iou = torch.tensor([]).cuda()
        self.recall = Recall()
        self.model = model
        self.treshold = treshold
        self.cosine_similarity = torch.tensor([]).cuda()
        self.euclidean_distance = torch.tensor([]).cuda()

    def update(self, predicted_bbox, target_bbox, predicted_image, target_image):
        predicted_bbox = torch.tensor(predicted_bbox)
        target_bbox = torch.tensor(target_bbox)

        with torch.no_grad():
            # Preprocess the predicted image and compute the predicted embedding
            predicted_image = padd_image(predicted_image)
            image_tensor = self.transform(predicted_image)
            image_tensor = image_tensor.unsqueeze(
                0).cuda()  # Add batch dimension
            predicted_embedding = self.model.encode_image(image_tensor)

            # Preprocess the target image and compute the target embedding
            target_image = padd_image(target_image)
            target_image_tensor = self.transform(target_image)
            target_image_tensor = target_image_tensor.unsqueeze(
                0).cuda()  # Add batch dimension
            target_embedding = self.model.encode_image(target_image_tensor)

        similarity = torch.nn.functional.cosine_similarity(
            predicted_embedding, target_embedding)
        distance = torch.nn.functional.pairwise_distance(
            predicted_embedding, target_embedding)

        # convert bboxes into torch tensors
        predicted_bbox = torch.tensor(predicted_bbox)
        target_bbox = torch.tensor(target_bbox)
        predicted_bbox = convert_yolo_bbox(predicted_bbox)
        actual_iou = ops.box_iou(predicted_bbox.unsqueeze(
            0).cuda(), target_bbox.unsqueeze(0).cuda())
        self.iou = torch.cat((self.iou, actual_iou), 0)
        # get iou value of the predicted bbox and the target bbox
        if actual_iou > self.treshold:
            self.recall.update(True)
        else:
            self.recall.update(False)
        self.cosine_similarity = torch.cat(
            (self.cosine_similarity, similarity), 0)
        self.euclidean_distance = torch.cat(
            (self.euclidean_distance, distance), 0)

    def to_string(self):
        mean_iou = torch.mean(self.iou)
        recall_at_05_iou = self.recall.compute()
        mean_cosine_similarity = torch.mean(self.cosine_similarity)
        mean_euclidean_distance = torch.mean(self.euclidean_distance)

        return f"Mean IoU: {mean_iou:.4f}, Recall@0.5 IoU: {recall_at_05_iou:.4f}, Mean Cosine Similarity: {mean_cosine_similarity:.4f}, Mean Euclidean Distance: {mean_euclidean_distance:.4f}"

    def reset(self):
        self.iou = torch.tensor([]).cuda()
        self.recall.reset()
        self.cosine_similarity = torch.tensor([]).cuda()
        self.euclidean_distance = torch.tensor([]).cuda()

### Dataset


In [38]:
# dataset class definition
class Coco(Dataset):
    def __init__(self, path_json, path_pickle, train=True):
        self.path_json = path_json
        self.path_pickle = path_pickle
        self.train = train

        # load images and annotations
        with open(self.path_json) as json_data:
            data = json.load(json_data)
            self.ann_frame = pd.DataFrame(data['annotations'])
            self.ann_frame = self.ann_frame.reset_index(drop=False)

        with open(self.path_pickle, 'rb') as pickle_data:
            data = pickle.load(pickle_data)
            self.refs_frame = pd.DataFrame(data)

        # separate each sentence in dataframe
        self.refs_frame = self.refs_frame.explode('sentences')
        self.refs_frame = self.refs_frame.reset_index(drop=False)

        self.size = self.refs_frame.shape[0]

        # merge the dataframes
        self.dataset = pd.merge(
            self.refs_frame, self.ann_frame, left_on='ann_id', right_on='id')
        # drop useless columns for cleaner and smaller dataset
        self.dataset = self.dataset.drop(columns=['segmentation', 'id', 'category_id_y', 'ref_id', 'index_x',
                                         'iscrowd', 'image_id_y', 'image_id_x', 'category_id_x', 'ann_id', 'sent_ids', 'index_y', 'area'])

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.dataset.iloc[idx]

    def get_annotation(self, idx):
        return self.ann_frame.iloc[idx]

    def get_imgframe(self, idx):
        return self.img_frame.iloc[idx]

    def get_validation(self):
        return self.dataset[self.dataset['split'] == 'val']

    def get_test(self):
        return self.dataset[self.dataset['split'] == 'test']

    def get_train(self):
        return self.dataset[self.dataset['split'] == 'train']

In [22]:
# dataset loading test and dimension check
dataset = Coco(local_annotations + 'instances.json',
               local_annotations + "refs(umd).p")

print(len(dataset))

95010


In [23]:
# to test prints the dataframe that composes the dataset
dataset.dataset

,split,sentences,file_name,bbox
0,test,"{'tokens': ['the', 'man', 'in', 'yellow', 'coa...",COCO_train2014_000000380440_491042.jpg,"[374.31, 65.06, 136.04, 201.94]"
1,test,"{'tokens': ['skiier', 'in', 'red', 'pants'], '...",COCO_train2014_000000380440_491042.jpg,"[374.31, 65.06, 136.04, 201.94]"
2,test,"{'tokens': ['there', 'is', 'red', 'colored', '...",COCO_train2014_000000419645_398406.jpg,"[93.95, 83.29, 504.61, 290.57]"
3,test,"{'tokens': ['a', 'shiny', 'red', 'vintage', 'p...",COCO_train2014_000000419645_398406.jpg,"[93.95, 83.29, 504.61, 290.57]"
4,test,"{'tokens': ['a', 'apple', 'desktop', 'computer...",COCO_train2014_000000478885_124383.jpg,"[338.8, 82.19, 147.34, 157.37]"
...,...,...,...,...
95005,train,"{'tokens': ['the', 'larger', 'banana', 'is', '...",COCO_train2014_000000003518_1042682.jpg,"[141.7, 20.68, 359.02, 144.57]"
95006,train,"{'tokens': ['a', 'guy', 'in', 'black', 'jacket...",COCO_train2014_000000302199_473946.jpg,"[141.34, 86.71, 115.33, 288.75]"
95007,train,"{'tokens': ['the', 'man', 'in', 'the', 'dark',...",COCO_train2014_000000302199_473946.jpg,"[141.34, 86.71, 115.33, 288.75]"
95008,train,"{'tokens': ['a', 'person', 'in', 'red', 'dress...",COCO_train2014_000000573297_472971.jpg,"[31.71, 67.03, 352.29, 572.97]"


### Model


In [69]:
similarities_values=[]

In [73]:
# class that defines the baseline model

class VisualGrounding(torch.nn.Module):
    def __init__(self, yolo_version, clip_version, local_path, img_path):
        super(VisualGrounding, self).__init__()
        self.local_path = local_path
        self.img_path = img_path
        # initialize models
        self.yolo = torch.hub.load(
            'ultralytics/yolov5', yolo_version, pretrained=True)
        self.clip, self.preprocess = clip.load(clip_version)

        # define metrics
        treshold = 0.5
        transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])
        ])
        self.metrics = Metrics(self.clip, transform, treshold)

    def forward(self, img_path, sentence,index):
        max_similarity = 0
        max_image = None
        max_bbox = None

        yolo_output = self.yolo(self.local_path+img_path)

        original_img = Image.open(self.local_path+img_path).convert("RGB")

        root = get_root(yolo_output, sentence, self.clip, self.yolo)


        stable_features=process_stable_images(index, self.clip)

        for i in range(len(yolo_output.xyxy[0])):
            #if root != "empty" and self.yolo.names[int(yolo_output.pred[0][i][5])] != root:
            #    continue
            x1 = yolo_output.xyxy[0][i][0].cpu().numpy()
            x1 = np.rint(x1)
            y1 = yolo_output.xyxy[0][i][1].cpu().numpy()
            y1 = np.rint(y1)
            x2 = yolo_output.xyxy[0][i][2].cpu().numpy()
            x2 = np.rint(x2)
            y2 = yolo_output.xyxy[0][i][3].cpu().numpy()
            y2 = np.rint(y2)

            img_cropped = original_img.crop((x1, y1, x2, y2))
            img_cropped=padd_image(img_cropped)
            img = self.preprocess(img_cropped).cuda().unsqueeze(0)
            #text = clip.tokenize([sentence]).cuda()

            with torch.no_grad():
                image_features = self.clip.encode_image(img).float()

            image_features /= image_features.norm(dim=-1, keepdim=True)
            stable_features /= stable_features.norm(dim=-1, keepdim=True)
            similarity = stable_features.cpu().numpy() @ image_features.cpu().numpy().T
            similarity=min(similarity)
            if similarity > max_similarity:
                max_similarity = similarity
                max_image = img_cropped
                max_bbox = yolo_output.xyxy[0][i]
        similarities_values.append(max_similarity)
        if max_image is None:
            #set bbox to the whole image
            max_bbox = [0, 0, original_img.width, original_img.height]
            max_image = original_img
            print("No bbox found for sentence: ", sentence)
            print(max_bbox)
            print("Image: ", max_image)

        return max_bbox, max_image

    def evaluate(self, img_path, sentence, gt, original_img,index):
        bbox = convert_bbox(gt, original_img)
        gt_crop = original_img.crop(bbox)
        prediction_bbox, prediction_img = self.forward(img_path, sentence,index)
        self.metrics.update(prediction_bbox, bbox, prediction_img, gt_crop)
        return prediction_bbox, prediction_img

    def reset_metrics(self):
        self.metrics.reset()

    def get_metrics(self):
        return self.metrics.to_string()

### Model evaluation


In [65]:
# dataset load
dataset = Coco(local_annotations + 'instances.json',
               local_annotations + "refs(umd).p")

In [74]:
# model load
model = VisualGrounding('yolov5x', 'ViT-B/32', local_path, local_annotations)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-5-30 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
# evaluation of the model
validate(model, dataset.get_validation())

Mean IoU: 0.5626, Recall@0.5 IoU: 0.5780, Mean Cosine Similarity: 0.8914, Mean Euclidean Distance: 4.3831


In [ ]:
validate(model, dataset.get_test())

In [72]:
similarities_values

[array([    0.48584], dtype=float32),
 array([    0.67357], dtype=float32),
 array([    0.72272], dtype=float32),
 array([    0.74439], dtype=float32),
 array([    0.66871], dtype=float32),
 array([    0.61215], dtype=float32),
 array([    0.64431], dtype=float32),
 array([    0.71681], dtype=float32),
 array([    0.40854], dtype=float32),
 array([    0.48027], dtype=float32),
 array([    0.62008], dtype=float32),
 array([    0.51312], dtype=float32),
 array([    0.76349], dtype=float32),
 array([    0.77909], dtype=float32),
 array([    0.60902], dtype=float32),
 array([    0.59823], dtype=float32),
 array([    0.59411], dtype=float32),
 array([    0.66313], dtype=float32),
 array([    0.68639], dtype=float32),
 array([    0.79853], dtype=float32),
 array([    0.50037], dtype=float32),
 array([    0.51484], dtype=float32),
 array([    0.56888], dtype=float32),
 array([    0.65997], dtype=float32),
 array([    0.58957], dtype=float32),
 array([    0.46174], dtype=float32),
 array([    